# Set-up

In [2]:
import os
import sys
import yaml
import logging
import mudata
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from typing import List, Dict, Tuple, Union, Optional, Literal, Mapping

from scipy.stats import pearsonr, spearmanr, kendalltau
from statsmodels.stats.multitest import multipletests

from tangermeme.io import read_meme, extract_loci
from tangermeme.tools.fimo import fimo

from joblib import Parallel, delayed
from tqdm.auto import tqdm

# Change path to wherever you have repo locally
sys.path.append('/cellar/users/aklie/opt/gene_program_evaluation')

from src.evaluation import (
    compute_categorical_association,
    compute_geneset_enrichment,
    compute_trait_enrichment,
    compute_perturbation_association,
    compute_explained_variance_ratio,
    compute_motif_enrichment
)
from src.evaluation.enrichment_trait import process_enrichment_data

In [3]:
# I/O paths
path_config = "/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF_30/evaluation_pipeline.yml"
config = yaml.safe_load(open(path_config))

## I/O

In [4]:
io_config = config['io']
io_config

{'path_mdata': '/cellar/users/aklie/opt/gene_program_evaluation/examples/inference/iPSC_EC/cNMF/cNMF_30_0.2_gene_names.h5mu',
 'path_out': '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF_30',
 'data_key': 'rna',
 'prog_key': 'cNMF'}

In [5]:
# Load mdata
path_mdata = io_config['path_mdata']
mdata = mudata.read(path_mdata)
mdata

/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/anndata/_core/anndata.py:522: FutureWarning: The dtype argument is deprecated and will be removed in late 2024.
  warnings.warn(
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


MuData object with n_obs × n_vars = 92284 × 5481
  2 modalities
    rna:	92284 x 5451
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
      obsm:	'X_pca', 'X_umap'
      layers:	'norm10k'
    cNMF:	92284 x 30
      obs:	'sample', 'species', 'gene_count', 'tscp_count', 'mread_count', 'leiden', 'n_counts'
      uns:	'guide_names', 'guide_targets', 'var_names'
      obsm:	'guide_assignment'
      varm:	'loadings'

In [6]:
prog_key = "cNMF"
data_key = "rna"

# Motif enrichment testing

In [11]:
motif_enrichment_config = config['motif_enrichment']
motif_enrichment_config

{'motif_file': '/cellar/users/aklie/opt/gene_program_evaluation/src/tests/test_data/motifs.meme',
 'seq_file': '/cellar/users/aklie/data/ref/genomes/hg38/hg38.fa',
 'loci_file': '/cellar/users/aklie/opt/gene_program_evaluation/src/tests/test_data/p2g_links_minimal_2024_09_02.txt',
 'output_loc': '/cellar/users/aklie/opt/gene_program_evaluation/examples/evaluation/iPSC_EC/cNMF_30/motif_counts',
 'use_previous': False,
 'sig': 0.05,
 'num_genes': None,
 'correlation': 'pearsonr',
 'n_jobs': -1,
 'inplace': False}

In [12]:
gene_names = mdata[prog_key].uns['var_names']
len(gene_names)

5451

## Read coordinate/bed/loci

In [13]:
def read_loci(
    path_loci: os.PathLike
) -> pd.DataFrame:
    """Read loci to gene links file

    Read promoter/enhancer to gene links tab delimited file with the following column headers:
        - chromosome: chromosome of the locus
        - start: start position of the locus
        - end: end position of the locus
        - seq_name: name of the locus
        - seq_class {promoter, enhancer}: class of the locus
        - seq_score: score of the locus
        - gene_name: name of the gene linked to the locus

    Parameters
    ----------
    path_loci : str
        Path to the coordinates file.

    Returns
    -------
    loci : pd.DataFrame
        DataFrame containing the coordinates.
    """

    # Read formatted coords file
    loci = pd.read_csv(path_loci, sep='\t')

    # Enforce header format
    expected_headers = [
        'chromosome', 
        'start', 
        'end', 
        'seq_name', 
        'seq_class', 
        'seq_score', 
        'gene_name'
    ]

    # Check for expected headers
    for col in expected_headers:
        try: assert col in loci.columns
        except: raise ValueError('Coordinate file is not formatted correctly')

    # TODO: Support multiple classes at once
    try: assert len(loci['seq_class']==1)
    except: raise ValueError('Coordinate file contains multiple sequence classes')

    return loci

In [14]:
motif_enrichment_config["coords_file"] = "/cellar/users/aklie/opt/gene_program_evaluation/src/tests/test_data/p2g_links_minimal_2024_09_02.txt"

In [15]:
loci = read_loci(motif_enrichment_config["coords_file"])
loci.head()

,chromosome,start,end,seq_name,seq_class,seq_score,gene_name
0,chr12,52299565,52301301,promoter|chr12:52299415-52301451,promoter,56.971568,ACVRL1
1,chr12,52299565,52301301,promoter|chr12:52299415-52301451,promoter,56.971568,ANKRD33
2,chr12,57028808,57030874,promoter|chr12:57028658-57031024,promoter,41.677500,BAZ2A
3,chr12,57028808,57030874,promoter|chr12:57028658-57031024,promoter,41.677500,ATP5B
4,chr12,89745534,89746814,promoter|chr12:89745384-89746964,promoter,43.190770,DUSP6


In [16]:
matching_gene_names = np.intersect1d(loci['gene_name'].unique(), gene_names)
len(matching_gene_names)

3

In [17]:
# Filter loci to only include gene names that are in the gene program
loci_= loci[loci['gene_name'].isin(matching_gene_names)]
loci_.head()

,chromosome,start,end,seq_name,seq_class,seq_score,gene_name
0,chr12,52299565,52301301,promoter|chr12:52299415-52301451,promoter,56.971568,ACVRL1
4,chr12,89745534,89746814,promoter|chr12:89745384-89746964,promoter,43.190770,DUSP6
9,chr13,21099338,21101201,promoter|chr13:21099188-21101351,promoter,40.740980,CRYL1


In [18]:
X = extract_loci(
    loci_,
    motif_enrichment_config["seq_file"], 
    in_window=1000
).float()
X.shape

torch.Size([3, 4, 1000])

## Read in motifs

In [19]:
pwms = read_meme(motif_enrichment_config['motif_file'])
list(pwms.keys())[:5]

['AHR.H12CORE.0.P.B',
 'AHRR.H12CORE.0.P.C',
 'ALX1.H12CORE.0.SM.B',
 'ALX3.H12CORE.0.SM.B',
 'FOXF1.H12CORE.0.P.C']

## Motif matching with FIMO

In [20]:
def perform_motif_match(
    loci: pd.DataFrame,
    sequences: os.PathLike,
    pwms: Mapping[str, np.ndarray],
    in_window: int=1000,
    threshold: float=1e-4,
    eps: float=1e-4,
    reverse_complement: bool=True,
    output_loc: os.PathLike=None
):
        """Score motif matches

        Perform motif matching on sequences linked to genes
        via enhancer/promoter coordinates.

        Parameters
        ----------
        loci : pd.DataFrame
            DataFrame containing sequence coordinates.
        sequences : os.PathLike
            Path to FASTA formatted genomic sequence.
        pwms : Mapping[str, np.ndarray]
            Dictionary of PWMs where keys are motif names and values are PWMs.
        in_window : int
            Window size to extract sequences around center of loci.
        threshold : float
            Threshold for motif matching.
        output_loc : os.PathLike
            Path to directory to store motif matches for individual motifs.
        """
        # Perform motif matching
        X = extract_loci(loci, sequences, in_window=in_window).float()
        hits = fimo(pwms, X, threshold=threshold, eps=eps, reverse_complement=reverse_complement)

        # Create motif match dataframe
        motif_match_df = pd.DataFrame()
        for i, hit in enumerate(hits):
            annotated_hit = hit.merge(loci[["chromosome", "seq_name", "seq_class", "gene_name"]], left_on="sequence_name", right_index=True).drop(columns=["sequence_name"])
            annotated_hit["motif_name"] = list(pwms.keys())[i]
            annotated_hit = annotated_hit[["chromosome", "start", "end", "strand", "motif_name", "score", "p-value", "seq_name", "seq_class", "gene_name"]]
            if output_loc is not None:
                annotated_hit.to_csv(os.path.join(output_loc, f"motif_match_{list(pwms.keys())[i]}.txt"), sep='\t', index=False)
            motif_match_df = pd.concat([motif_match_df, annotated_hit])
        motif_match_df["adj_pval"] = multipletests(motif_match_df["p-value"], method="fdr_bh")[1]
        motif_match_df.reset_index(drop=True, inplace=True)
        return motif_match_df

In [21]:
motif_match_df = perform_motif_match(loci_, motif_enrichment_config["seq_file"], pwms, in_window=1000, threshold=1e-4)
motif_match_df.head()

,chromosome,start,end,strand,motif_name,score,p-value,seq_name,seq_class,gene_name,adj_pval
0,chr12,405,425,-,ALX1.H12CORE.0.SM.B,9.742607,0.000076,promoter|chr12:52299415-52301451,promoter,ACVRL1,0.000076
1,chr12,443,455,-,FOXF1.H12CORE.0.P.C,11.801019,0.000068,promoter|chr12:52299415-52301451,promoter,ACVRL1,0.000076


## Compute counts of motifs for each gene

In [22]:
def compute_motif_instances(
    motif_match_df: pd.DataFrame,
    motif_var: str = 'motif_name',
    sig: float=0.05,
    sig_var: str='adj_pval',
    gene_names: Optional[np.ndarray]=None
):
    """Count motif instances per gene (via enahncer/promoter linking)

    Parameters
    ----------
    motif_match_df : pd.DataFrame
        DataFrame containing motif matches.
    motif_var : str
        Column name for motif names. Default is 'motif_name'.
    sig : float
        Significance threshold for motif matches. Default is 0.05.
    sig_var : str
        Column name for significance values. Default is 'adj_pval'.
    gene_names : np.ndarray
        Array of gene names. Default is None.
    
    Returns
    -------
    motif_count_df : pd.DataFrame
        DataFrame containing
    """

    # Count up significant occurences of motif
    motif_match_df_ = motif_match_df.loc[motif_match_df[sig_var] < sig]
    motif_match_df_ = motif_match_df.value_counts(subset=['gene_name', motif_var]).reset_index()
    motif_match_df_.columns = ['gene_name', motif_var, 'motif_count']
    motif_match_df_ = motif_match_df_.pivot(index='gene_name', columns=motif_var, values='motif_count')
    motif_count_df = pd.DataFrame(index=gene_names, columns=motif_match_df_.columns)
    motif_count_df.loc[motif_match_df_.index.values] = motif_match_df_ # Gene names should match as this point
    return motif_count_df

In [23]:
motif_count_df = compute_motif_instances(
    motif_match_df, 
    motif_var='motif_name', 
    sig=0.05, 
    sig_var='adj_pval', 
    gene_names=gene_names
)
motif_count_df.sum(axis=0)

motif_name
ALX1.H12CORE.0.SM.B    1
FOXF1.H12CORE.0.P.C    1
dtype: object

## Motif enrichment statistics

In [24]:
def perform_correlation(
    motif_count_df: pd.DataFrame,
    prog_genes: pd.DataFrame,
    motif_enrich_stat_df: pd.DataFrame,
    motif_enrich_pval_df: pd.DataFrame,
    motif_idx: int,
    prog_idx: int,
    correlation: str='pearsonr'
):
    """Compute motif enrichment as correlation b/w gene weights/ranks and motif counts
    
    Perform pearson correlation test for motif count enrichment vs gene loadings
    If loadings are dichotomized then this is equivalent to a point biserial correlation test.

    Parameters
    ----------
    motif_count_df : pd.DataFrame
        DataFrame containing motif counts.
    prog_genes : pd.DataFrame
        DataFrame containing gene program loadings.
    motif_enrich_stat_df : pd.DataFrame
        DataFrame to store correlation statistics.
    motif_enrich_pval_df : pd.DataFrame
        DataFrame to store correlation p-values.
    motif_idx : int
        Index of motif.
    prog_idx : int
        Index of gene program.
    correlation : {'pearsonr','spearmanr','kendalltau'}
        Type of correlation to perform. Default is 'pearsonr'.
    """

    loadings = prog_genes.iloc[prog_idx].values.flatten()
    counts = motif_count_df.T.iloc[motif_idx].fillna(0).values.flatten()
    
    if correlation=='pearsonr':
        stat, pval = pearsonr(loadings, counts)
    elif correlation=='spearmanr':
        stat, pval = spearmanr(loadings, counts)
    elif correlation=='kendalltau':
        stat, pval = kendalltau(loadings, counts)

    motif_enrich_stat_df.iloc[prog_idx, motif_idx]  = stat
    motif_enrich_pval_df.iloc[prog_idx, motif_idx]  = pval


def compute_motif_enrichment_(
    mdata: mudata.MuData,
    motif_count_df: pd.DataFrame,
    prog_key: str='prog',
    gene_names: Optional[np.ndarray]=None,
    weighted: bool=True,
    num_genes: Optional[int]=None,
    correlation: str='pearsonr',
    n_jobs: int=1
):
    """Count up motif ocurrences and perform diff. test

    Perform motif enrichment using gene program loadings and
    motif counts linked to genes via motif scanning of
    linked enhancer/promoter sequences.
    
    Parameters
    ----------
    mdata : MuData
        MuData object containing anndata of program scores and cell-level metadata.
    motif_count_df : pd.DataFrame
        DataFrame containing motif counts.
    prog_key : str
        Key for the anndata object in the mudata object. Default is 'prog'.
    gene_names : np.ndarray
        Array of gene names. Default is None.
    weighted : bool
        Use weighted loadings. Default is True.
    num_genes : int
        Number of genes threshold to dichotomize loadings. Default is None.
    correlation : {'pearsonr','spearmanr','kendalltau'}
        Type of correlation to perform. Default is 'pearsonr'.
    n_jobs : int
        Number of threads to run processes on. Default is 1.
    """

    # Both weighted and num_genes cannot be set
    if num_genes is not None and weighted:
        raise ValueError('Will not use weighted when num_genes specified.')

    loadings = pd.DataFrame(data=mdata[prog_key].varm['loadings'],
                            index=mdata[prog_key].var_names,
                            columns=gene_names)
    # FIXME: Causes expansion due to duplications in gene_names
    # Ensure index matches b/w loadings and counts
    # loadings = loadings.loc[:, motif_count_df.index.values]

    # Binary matrix 
    if not weighted:
        prog_genes = (loadings.rank(axis=1)<num_genes).astype(int)
    elif weighted and num_genes is None:
        prog_genes = loadings
    elif weighted and isinstance(num_genes, int):
        prog_genes = (loadings.rank(axis=1)<num_genes).astype(int)
        prog_genes *= loadings
    else:
        raise ValueError('num_genes not specified correctly.')

    # Use pearson correlation
    # If dichotomized then this is equivalent to point biserial correlation
    motif_enrich_stat_df = pd.DataFrame(index=mdata[prog_key].var_names,
                                        columns=motif_count_df.columns.values)
    motif_enrich_pval_df = pd.DataFrame(index=mdata[prog_key].var_names,
                                        columns=motif_count_df.columns.values)

    # FIXME: If n_jobs>1 then parallel processes dont seem to terminate.
    # Perform test in parallel across motifs and programs
    Parallel(n_jobs=n_jobs, 
             backend='threading')(delayed(perform_correlation)(motif_count_df,
                                                               prog_genes, 
                                                               motif_enrich_stat_df,
                                                               motif_enrich_pval_df,
                                                               motif_idx, 
                                                               prog_idx,
                                                               correlation=correlation) \
                                                            for motif_idx in tqdm(range(motif_count_df.columns.values.shape[-1]),
                                                                                     desc='Computing motif enrichment',
                                                                                     unit='motifs') \
                                                            for prog_idx in range(mdata[prog_key].var_names.shape[0]))

    return motif_enrich_stat_df, motif_enrich_pval_df

In [25]:
motif_enrich_stat_df, motif_enrich_pval_df = compute_motif_enrichment_(
    mdata,
    motif_count_df=motif_count_df,
    prog_key=prog_key,
    gene_names=gene_names,
    num_genes=None,
    n_jobs=1,
)

Computing motif enrichment:   0%|          | 0/2 [00:00<?, ?motifs/s]

# Wrapper

In [26]:
def compute_motif_enrichment(
    mdata: Union[mudata.MuData, os.PathLike],
    prog_key: str='prog',
    data_key: str='data',
    motif_file: Optional[os.PathLike]=None,
    seq_file: Optional[os.PathLike]=None,
    loci_file: Optional[os.PathLike]=None,
    output_loc: Optional[os.PathLike]=None,
    window: int=1000,
    threshold: float=1e-4,
    eps: float=1e-4,
    reverse_complement: bool=True,
    sig: float=0.05,
    num_genes: Optional[int]=None,
    correlation: str='pearsonr',
    n_jobs: int=1,
    inplace: bool=True,
    **kwargs
):
    
    """Compute motif enrichment in enhancers or promoters associated with a gene
    
    Perform motif enrichment using gene program loadings and
    motif counts linked to genes via motif scanning of
    linked enhancer/promoter sequences.

    Parameters
    ----------
    mdata : MuData
        mudata object containing anndata of program scores and cell-level metadata.
    prog_key: 
        index for the anndata object (mdata[prog_key]) in the mudata object.
    data_key: str
        index of the genomic data anndata object (mdata[data_key]) in the mudata object.
    motif_file: str
        path to motif file formatted in MEME format.
    seq_file: str
        path to FASTA formatted genomic sequence.
    loci_file: str
        path to enhancer/promoter gene links file with sequence coordinates.
        Tab delimited file with the following column headers:
        chr, start, end, seq_name, seq_class {promoter, enhancer}, 
        seq_score, gene_name.
    output_loc: str
        path to directory to store motif - gene counts.
    sig: (0,1] (default: 0.05)
        significance level for inferring a motif match.
    num_genes: int (default: None)
        number of genes threshold to dichtomize loadings.
    correlation: {'pearsonr','spearmanr','kendalltau'} (default: 'peasronsr')
        correlation type to use to compute motif enirchments.
        Use kendalltau when expecting enrichment/de-enrichment at both ends.
    use_previous: bool (default: True)
        if outplot is provided try to load motif matches from previous run.
    n_jobs: int (default: 1)
        number of threads to run processes on.
    inplace: Bool (default: True)
        update the mudata object inplace or return a copy
       
    Returns
    -------
    if not inplace:
        motif_match_df,
        motif_count_df.loc[gene_names].values,
        motif_enrichment_df
    else:
        None, edits mdata in place
    """

    # Read in mudata if it is provided as a path
    frompath=False
    if isinstance(mdata, str):
        if os.path.exists(mdata):
            mdata = mudata.read(mdata)
            if inplace:
                logging.warning('Changed to inplace=False since path was provided')
                inplace=False
            frompath=True
        else: raise ValueError('Incorrect mudata specification.')
    
    if not inplace and not frompath:
        mdata = mudata.MuData({prog_key: mdata[prog_key].copy(),
                               data_key: mdata[data_key].copy()})

    # Get gene names in MuData
    if 'var_names' in mdata[prog_key].uns.keys():
        gene_names = mdata[prog_key].uns['var_names']
    else:
        try: assert mdata[prog_key].varm['loadings'].shape[1]==mdata[data_key].var.shape[0]
        except: raise ValueError('Different number of genes present in data and program loadings')
        gene_names = mdata[data_key].var_names
    
    # 
    if ':ens' in gene_names[0].lower():
        gene_names = [name.split(':')[0] for name in gene_names]

    # Check if output loc exists
    if output_loc is not None:
        try: os.makedirs(output_loc, exist_ok=True)
        except: raise ValueError('Output location does not exist.')

    # If num_genes specified then cannot be weighted
    if num_genes is None:
        weighted=True
    elif isinstance(num_genes, int):
        weighted=False

    # Intake motif file path or in memory
    if isinstance(motif_file, str) and os.path.exists(motif_file):
        pwms = read_meme(motif_file)
    else:
        raise ValueError('Motif file not found.')
    
    # Intake coord file path or in memory
    if isinstance(loci_file, str) and os.path.exists(loci_file):
        loci = read_loci(loci_file)
    else:
        raise ValueError('Coordinate file not found.')

    # Valid genes
    matching_gene_names = np.intersect1d(loci['gene_name'].unique(), gene_names)
    print(f'Number of matching genes: {len(matching_gene_names)}')
    try: assert len(matching_gene_names) > 0
    except: raise ValueError('No matching genes b/w data and coordinate files')

    # Compute motif matching
    loci_ = loci[loci['gene_name'].isin(matching_gene_names)]
    print(f'Number of loci: {loci_.shape[0]}')
    motif_match_df = perform_motif_match(
        loci=loci_,
        sequences=seq_file,
        pwms=pwms,
        in_window=window,
        threshold=threshold,
        eps=eps,
        reverse_complement=reverse_complement,
        output_loc=output_loc
    )

    # Count motif enrichment
    motif_count_df = compute_motif_instances(
        motif_match_df,
        motif_var='motif_name',
        sig=sig,
        sig_var='adj_pval',
        gene_names=gene_names
    )
    motif_enrich_stat_df, motif_enrich_pval_df = compute_motif_enrichment_(
        mdata,
        motif_count_df,
        prog_key=prog_key,
        gene_names=gene_names,
        weighted=weighted,
        num_genes=num_genes,
        n_jobs=n_jobs)
                         
    # Store motif counts
    if inplace:
        mdata[prog_key].uns['motif_counts'] = motif_count_df.loc[gene_names].values
        mdata[prog_key].uns['motif_names'] = motif_count_df.columns.values

        mdata[prog_key].varm['motif_enrich_{}_stat'.format(correlation)] = motif_enrich_stat_df.values
        mdata[prog_key].varm['motif_enrich_{}_pval'.format(correlation)] = motif_enrich_pval_df.values
        mdata[prog_key].uns['motif_names'] = motif_count_df.columns.values

    else:

        motif_enrich_stat_df = motif_enrich_stat_df.reset_index().melt(id_vars='index',
                                                                       var_name='motif', 
                                                                       value_name='stat')
        motif_enrich_stat_df = motif_enrich_stat_df.set_index(['index', 'motif'])

        motif_enrich_pval_df = motif_enrich_pval_df.reset_index().melt(id_vars='index',
                                                                       var_name='motif', 
                                                                       value_name='pval')
        motif_enrich_pval_df = motif_enrich_pval_df.set_index(['index', 'motif'])

        motif_enrichment_df = motif_enrich_stat_df.merge(motif_enrich_pval_df,
                                                        left_index=True, 
                                                        right_index=True)
        motif_enrichment_df = motif_enrichment_df.reset_index()
        motif_enrichment_df['program_name'] = motif_enrichment_df['index']
        motif_enrichment_df.drop('index', axis=1, inplace=True)
        motif_enrichment_df = motif_enrichment_df.sort_values(['program_name', 'pval'])

        motif_count_df.columns.name=''

        return (motif_match_df,
                motif_count_df.loc[gene_names].fillna(0),
                motif_enrichment_df)

In [27]:
# Run motif enrichment and save results
motif_match_minimal_df, motif_count_minimal_df, motif_enrichment_minimal_df = compute_motif_enrichment(
    mdata, 
    prog_key=prog_key,
    data_key=data_key,
    motif_file=motif_enrichment_config['motif_file'],
    seq_file=motif_enrichment_config['seq_file'],
    loci_file="/cellar/users/aklie/opt/gene_program_evaluation/src/tests/test_data/p2g_links_minimal_2024_09_02.txt",
    output_loc=None,
    window=1000,
    threshold=1e-4,
    eps=1e-4,
    reverse_complement=True,
    sig=0.05,
    num_genes=None,
    correlation='pearsonr',
    n_jobs=1,
    inplace=False
)

/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


Number of matching genes: 3
Number of loci: 3


Computing motif enrichment:   0%|          | 0/2 [00:00<?, ?motifs/s]

In [29]:
motif_enrichment_minimal_df

,motif,stat,pval,program_name
0,ALX1.H12CORE.0.SM.B,-0.007562,0.576735,0
30,FOXF1.H12CORE.0.P.C,-0.007562,0.576735,0
1,ALX1.H12CORE.0.SM.B,0.007341,0.587885,1
31,FOXF1.H12CORE.0.P.C,0.007341,0.587885,1
10,ALX1.H12CORE.0.SM.B,0.054421,0.000058,10
40,FOXF1.H12CORE.0.P.C,0.054421,0.000058,10
11,ALX1.H12CORE.0.SM.B,-0.006529,0.629873,11
41,FOXF1.H12CORE.0.P.C,-0.006529,0.629873,11
12,ALX1.H12CORE.0.SM.B,0.027282,0.043993,12
42,FOXF1.H12CORE.0.P.C,0.027282,0.043993,12


In [30]:
motif_match_minimal_df

,chromosome,start,end,strand,motif_name,score,p-value,seq_name,seq_class,gene_name,adj_pval
0,chr12,405,425,-,ALX1.H12CORE.0.SM.B,9.742607,0.000076,promoter|chr12:52299415-52301451,promoter,ACVRL1,0.000076
1,chr12,443,455,-,FOXF1.H12CORE.0.P.C,11.801019,0.000068,promoter|chr12:52299415-52301451,promoter,ACVRL1,0.000076


In [26]:
# Run motif enrichment and save results
motif_match_D0_df, motif_count_D0_df, motif_enrichment_D0_df = compute_motif_enrichment(
    mdata, 
    prog_key=prog_key,
    data_key=data_key,
    motif_file=motif_enrichment_config['motif_file'],
    seq_file=motif_enrichment_config['seq_file'],
    loci_file="/cellar/users/aklie/opt/gene_program_evaluation/examples/datasets/iPSC_EC/EnhancerPredictions_D0_2024_09_07.txt",
    output_loc=None,
    window=1000,
    threshold=1e-4,
    eps=1e-4,
    reverse_complement=True,
    sig=0.05,
    num_genes=None,
    correlation='pearsonr',
    n_jobs=1,
    inplace=False
)

/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/cellar/users/aklie/opt/miniconda3/envs/test_celloracle/lib/python3.10/site-packages/mudata/_core/mudata.py:479: UserWarning: var_names are not unique. To make them unique, call `.var_names_make_unique`.
  warnings.warn(


Number of matching genes: 4420
Number of loci: 15692


Computing motif enrichment:   0%|          | 0/8 [00:00<?, ?motifs/s]

In [28]:
motif_match_D0_df

,chromosome,start,end,strand,motif_name,score,p-value,seq_name,seq_class,gene_name,adj_pval
0,chr16,607,617,+,AHR.H12CORE.0.P.B,14.323496,0.000006,genic|chr16:2715057-2715968,Enchancer,PRSS27,0.000073
1,chr16,611,621,+,AHR.H12CORE.0.P.B,14.598420,0.000004,genic|chr16:2715057-2715968,Enchancer,PRSS27,0.000073
2,chr16,57,67,-,AHR.H12CORE.0.P.B,14.376087,0.000006,genic|chr16:2715057-2715968,Enchancer,PRSS27,0.000073
3,chr16,542,552,-,AHR.H12CORE.0.P.B,12.360946,0.000037,genic|chr16:2715057-2715968,Enchancer,PRSS27,0.000087
4,chr16,287,297,+,AHR.H12CORE.0.P.B,11.207113,0.000088,intergenic|chr16:2905249-2906117,Enchancer,PRSS27,0.000096
...,...,...,...,...,...,...,...,...,...,...,...
5652,chr17,89,105,-,TEAD4.H12CORE.1.P.B,13.177800,0.000020,intergenic|chr17:427213-427713,Enchancer,C17orf97,0.000080
5653,chr17,818,834,-,TEAD4.H12CORE.1.P.B,12.383566,0.000034,genic|chr17:2023558-2024058,Enchancer,RTN4RL1,0.000086
5654,chr17,710,726,-,TEAD4.H12CORE.1.P.B,11.962195,0.000045,genic|chr17:4895716-4896445,Enchancer,C17orf107,0.000089
5655,chr17,318,334,-,TEAD4.H12CORE.1.P.B,12.134159,0.000039,intergenic|chr17:7280453-7280953,Enchancer,EIF5A,0.000088


# DONE!

---